<a href="https://colab.research.google.com/github/ayobamiakomolafe/Zindi-projects/blob/master/UmojaHack_Nigeria_AXA_Vehicle_Insurance_Claim_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Can you predict if a client will submit a vehicle insurance claim in the next 3 months?**

In [1]:
Train='/content/drive/MyDrive/Train.csv'
Test='/content/drive/MyDrive/Test.csv'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
df_train=pd.read_csv(Train)
df_test=pd.read_csv(Test)

In [4]:
df=pd.concat([df_train, df_test], axis=0)

Fill missing values with 999, Extract Year and Month from Datetime columns and Label encode categorical features

In [5]:
df.fillna(999, inplace=True)

In [6]:
df['Policy Start Date']=df_train['Policy Start Date'].astype('datetime64')
df['Policy End Date']=df_train['Policy End Date'].astype('datetime64')
df['First Transaction Date']=df_train['First Transaction Date'].astype('datetime64')

In [7]:
df['Policy Start Date Year']= df['Policy Start Date'].dt.year
df['Policy Start Date Month']= df['Policy Start Date'].dt.month
df['Policy End Date Year']= df['Policy End Date'].dt.year
df['Policy End Date Month']=df['Policy End Date'].dt.month
df['First Transaction Date Year']= df['First Transaction Date'].dt.year
df['First Transaction Date Month']= df['First Transaction Date'].dt.month


In [8]:
df.drop(['Policy Start Date','Policy End Date','First Transaction Date'], axis=1, inplace=True)

In [9]:
for i in ['Entity','Joint Gender' ,'NOT STATED','NO GENDER','SEX']:
    df['Gender'].replace(i, 'others', inplace=True)

In [ ]:
df.info()

In [11]:
state=dict()
nu=df['State'].nunique()
num=range(nu)
uni=df['State'].unique()
for a,b in zip(num,uni):
    df['State'].replace(b,a, inplace=True)
    state[b]=a

In [12]:
ProductName=dict()
nu=df['ProductName'].nunique()
num=range(nu)
uni=df['ProductName'].unique()
for a,b in zip(num,uni):
    df['ProductName'].replace(b,a, inplace=True)
    ProductName[b]=a


LGA_Name=dict()
nu=df['LGA_Name'].nunique()
num=range(nu)
uni=df['LGA_Name'].unique()
for a,b in zip(num,uni):
    df['LGA_Name'].replace(b,a, inplace=True)
    LGA_Name[b]=a

Subject_Car_Make=dict()
nu=df['Subject_Car_Make'].nunique()
num=range(nu)
uni=df['Subject_Car_Make'].unique()
for a,b in zip(num,uni):
    df['Subject_Car_Make'].replace(b,a, inplace=True)
    Subject_Car_Make[b]=a

In [13]:
Car_Category=dict()
nu=df['Car_Category'].nunique()
num=range(nu)
uni=df['Car_Category'].unique()
for a,b in zip(num,uni):
    df['Car_Category'].replace(b,a, inplace=True)
    Car_Category[b]=a

Subject_Car_Colour=dict()
nu=df['Subject_Car_Colour'].nunique()
num=range(nu)
uni=df['Subject_Car_Colour'].unique()
for a,b in zip(num,uni):
    df['Subject_Car_Colour'].replace(b,a, inplace=True)
    Subject_Car_Colour[b]=a

In [14]:
dg=pd.get_dummies(df['Gender'])
df=pd.concat([df,dg], axis=1)

In [15]:
df.drop(['Gender', 'ID'], axis=1, inplace=True)

In [ ]:
df.info()

In [39]:
df.head()

,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target,Policy Start Date Year,Policy Start Date Month,Policy End Date Year,Policy End Date Month,First Transaction Date Year,First Transaction Date Month,999,Female,Male,others
0,30,1,0,0,0,0,0,0,0.0,2010,5,2011,5,2010,5,0,0,1,0
1,79,1,1,1,0,0,0,0,1.0,2010,11,2011,11,2010,11,0,1,0,0
2,43,1,0,2,0,0,0,0,0.0,2010,3,2011,3,2010,3,0,0,1,0
3,2,1,2,3,1,0,0,1,0.0,2010,8,2011,8,2010,8,0,0,1,0
4,20,3,2,3,1,1,1,2,1.0,2010,8,2010,12,2010,8,0,0,0,1


In [17]:
df_train=df.iloc[:12079]
df_test=df.iloc[12079:]

In [18]:
df_X=df_train.drop('target', axis=1)
df_Y=df_train['target']

In [19]:
df_test=df_test.drop('target', axis=1)

In [ ]:
df_test.shape

Modelling with VotingClassifier

In [ ]:
!pip install lightgbm
!pip install catboost

In [24]:

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
lg=LGBMClassifier()
cat=CatBoostClassifier()
Rf=RandomForestClassifier()
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
scale=StandardScaler()

In [26]:
vf=VotingClassifier([('Rf', Rf), ('lg', lg), ('cat', cat)], voting='soft', weights=[0.7, 0.2,0.1])

In [27]:
df_X_sc=scale.fit_transform(df_X)
df_test_sc=scale.transform(df_test)


In [ ]:
vf.fit(df_X_sc,df_Y )

In [29]:
pred=vf.predict_proba(df_test_sc)

In [30]:
pred=pred[:,1]

In [31]:
final_pred=list()
for i in pred:
    x=None
    if i >= 0.2:
        x=1
    else:
        x=0
    final_pred.append(x)


In [32]:
final_pred[0:5]

[1, 0, 1, 0, 1]

In [33]:
pred=pd.DataFrame(final_pred)
pred.head()

,0
0,1
1,0
2,1
3,0
4,1


In [34]:
dfid=pd.read_csv('/content/drive/MyDrive/Test.csv')
dfid=dfid[['ID']]

In [35]:
df_sub=pd.concat([dfid,pred], axis=1)

In [36]:
df_sub.head()

,ID,0
0,009D84L,1
1,01DO2EQ,0
2,01QM0NU,1
3,024NJLZ,0
4,02BYET3,1


In [37]:
from google.colab import files
df_sub.to_csv('./ok4.csv', sep = ',', index = False)

In [38]:
files.download('ok4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>